In [30]:
import pandas as pd

emissions_df = pd.read_excel('../data//LAEI2013_Emissions_Summary-NOx_v1.1.xlsx', sheet_name='NOx by Grid Exact Cut',
                             skiprows=3)

nox_concentrations2013_df = pd.read_csv('../data/PostLAEI2013_2013_NOx.csv')
nox_concentrations2020_df = pd.read_csv('../data/2020_Met2013_LAEI2013_NOx.csv')
nox_concentrations2025_df = pd.read_csv('../data/2025_Met2013_LAEI2013_NOx.csv')
nox_concentrations2030_df = pd.read_csv('../data/2030_Met2013_LAEI2013_NOx.csv')

concentrations_df = pd.concat([nox_concentrations2013_df,
                               nox_concentrations2020_df,
                               nox_concentrations2025_df,
                               nox_concentrations2030_df],
                              ignore_index=True)

concentrations_df.head()
print(concentrations_df.isnull().sum())

emissions_df.columns = emissions_df.columns.str.replace(' ', '_')

concentrations_df = concentrations_df.rename(columns={
    'conct': 'NOx_concentration',
    'x': 'Easting',
    'y': 'Northing',
    'year': 'Year'
})

merged_df = pd.merge(concentrations_df, emissions_df, on=['Easting', 'Northing', 'Year'], how='inner')

print("Merged Data:")
display(merged_df)
merged_df.to_csv("../data/concentrations_merged_output.csv")

x        0
y        0
conct    0
year     0
dtype: int64
Merged Data:


,Easting,Northing,NOx_concentration,Year,LAEI_1km2_ID,GRID_ExactCut_ID,Borough,Zone,Household_and_Garden,Small_Waste_and_Accidental_Fires,...,Diesel_Car,Electric_Car,Petrol_LGV,Diesel_LGV,Electric_LGV,TfL_Bus,Non-TfL_Bus_and_Coach,Artic_HGV,Rigid_HGV,Total
0,501500,170500,56.87044,2013,11577,258,NonGLA,NonGLA,0.023479,0.002008,...,17.483541,0,0.135210,6.173631,0,0.167362,2.005301,6.539005,4.647952,43.364595
1,501500,187500,56.68350,2013,8653,224,NonGLA,NonGLA,0.011739,0.000367,...,14.099647,0,0.117319,5.491909,0,0.126726,0.882398,6.422596,3.777841,35.585632
2,502500,167500,33.61423,2013,12094,288,NonGLA,NonGLA,0.005870,0.000187,...,12.429571,0,0.101858,5.498087,0,0.186480,0.860553,4.819511,3.294308,32.519997
3,502500,168500,31.32178,2013,11922,275,NonGLA,NonGLA,0.011739,0.010782,...,0.647922,0,0.003241,0.255436,0,0.024594,0.027844,0.084129,0.296846,3.355043
4,502500,169500,32.42097,2013,11750,265,NonGLA,NonGLA,0.012473,0.003986,...,0.401891,0,0.002391,0.183494,0,0.051577,0.055494,0.017542,0.093720,2.728494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12859,560500,186500,14.64165,2030,8884,2774,Havering,Outer,0.000000,0.000000,...,0.029694,0,0.000140,0.008731,0,0.000000,0.001468,0.000182,0.001501,0.278572
12860,560500,187500,14.52616,2030,8712,1403,NonGLA,NonGLA,0.001488,0.000039,...,0.028266,0,0.000121,0.006810,0,0.000303,0.000664,0.000189,0.001556,0.182452
12861,560500,187500,14.52616,2030,8712,1404,Havering,Outer,0.000839,0.000022,...,0.028589,0,0.000136,0.007853,0,0.000000,0.001319,0.000297,0.001838,0.151122
12862,561500,185500,14.48234,2030,9057,2133,NonGLA,NonGLA,0.000000,0.000000,...,0.000924,0,0.000002,0.000207,0,0.000000,0.000000,0.000000,0.000000,0.038504


In [31]:
### Training the model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

vehicle_columns = [
    'Year', 'Motorcycle', 'Taxi', 'Petrol_Car', 'Diesel_Car', 'Electric_Car',
    'Petrol_LGV', 'Diesel_LGV', 'Electric_LGV', 'TfL_Bus', 'Non-TfL_Bus_and_Coach',
    'Artic_HGV', 'Rigid_HGV', 'Resuspension', 'Rail_Passengers', 'Rail_Freight', 'Commercial_Shipping',
    'Passenger_Shipping', 'Aviation', 'Industry_Part_B', 'Industry_Part_A', 'Commercial_Other_Fuels',
    'Domestic_Other_Fuels', 'Commercial_Gas', 'Domestic_Gas', 'C&D_Dust', 'NRMM_Industry',
    'NRMM_Construction', 'Landfill', 'WTS', 'STW', 'Agriculture', 'Small_Waste_and_Accidental_Fires',
    'Household_and_Garden',
]
features = merged_df[vehicle_columns]
target = merged_df['NOx_concentration']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 128.5892248556574
